In [2]:
from sklearn.model_selection import cross_val_score

from sklearn.datasets import load_iris

from sklearn.linear_model import LogisticRegression

# 交差検証
交差検証(cross-validation)はデータの分割を何度も繰り返して行い，複数のモデルを訓練する．  
訓練セットとテストセットの分割を何度もするイメージに近い．  

### k-fold cross-validation_
最もよく使われる交差検証はk-fold cross-validation．  
5-fold cross-validationはデータを5つに分割し1をテストセット，他を訓練セットして使用．   
これを1から5まで5回繰り返す．  

データセットの偏りに精度が依存する場合の性能の違いがわかる．  
分割数を増やすと，毎回の訓練データに使用できるデータ量が増える．  
ただし，交差検証はk倍時間がかかる．

また，データセットの正解ラベルの偏りを無くすために，stratified k-fold cross-validationがある．

In [4]:
# irisデータセットのk-fold cross-validation

iris = load_iris()
logreg = LogisticRegression(max_iter=1000)

scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print("Cross-validation scores: {}".format(scores))

print("Average cross-validation score: {:.2f}".format(scores.mean()))

Cross-validation scores: [0.96666667 1.         0.93333333 0.96666667 1.        ]
Average cross-validation score: 0.97


In [ ]:
# irisデータセットのstratified k-fold cross-validation

